# Mini Project Kelompok 4 Pendeteksi Kesehatan Cabai

# Memasukkan Data

In [1]:
import numpy as np
import matplotlib.pyplot as plt
import os 
import cv2


#memasukkan data
DATADIR = 'D:\Studi Independetn Orbit\TA\Data Baru\Latih-Cropped'
CATEGORIES = ['DaunSehat', 'Sakit']

for category in CATEGORIES:
    path = os.path.join(DATADIR, category) #path datanya
    for img in os.listdir(path): #mengambil semua gambar
        img_array = cv2.imread(os.path.join(path, img))#ini untuk mengubah image menjadi array menjadi array
        plt.imshow(img_array)
        plt.show()
        print(img_array.shape)
        break
    break

FileNotFoundError: ignored

# Melakukan Scaling Data Gambar

In [ ]:
IMG_SIZE = 250 #kita mengubah resolusi menjadi 250x250 (pixel)

new_array = cv2.resize(img_array, (IMG_SIZE, IMG_SIZE))
plt.imshow(new_array)
plt.show()

In [ ]:
#membuat training data
training_data = []

def create_training_data(CATEGORIES, DATADIR, app, IMG_SIZE):
    for category in CATEGORIES:
        path = os.path.join(DATADIR, category) #os.path.join() adalah function untuk menggabungkan directory
        
        #mengubah jadi 1 dan 0
        class_num = CATEGORIES.index(category) #ini mengambil nilai index dog dan cat
        
        for img in os.listdir(path):
            try: #try untuk mencegah ketika ada error yang muncul 
                img_array = cv2.imread(os.path.join(path, img))
                new_array = cv2.resize(img_array, (IMG_SIZE, IMG_SIZE))
                app.append([new_array, class_num]) #disini kita menyimpan array dan class_num
            
            except Exception as e:#ketika ada pemberitahuan kaya error maka di biarkan pass aja
                pass
            
create_training_data(CATEGORIES, DATADIR, training_data, 250)

In [ ]:
len(training_data)

# Melakukan Shuffle Untuk Data Training

In [ ]:
import random
random.shuffle(training_data)

#disini kita membagi menjadi x dan y nya 
x = [] #training data
y = [] #labelnya

#memasukkan feature dan label kedalam x dan y
for features, label in training_data:
    x.append(features)
    y.append(label)

#disini kita perlu mengubah menjadi array karena kalau list enggak bisa masuk ke API Keras
x = np.array(x).reshape(-1, IMG_SIZE, IMG_SIZE, 3) 

In [ ]:
x.shape

# Pemrosesan Data

In [ ]:
import tensorflow as tf
from tensorflow.keras.datasets import cifar10
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout, Activation, Flatten
from tensorflow.keras.layers import Conv2D, MaxPooling2D
import pickle
import numpy as np
from tensorflow.keras.callbacks import TensorBoard
import time

NAME = 'Tanaman-sehat-dan-sakit-cnn-64x2-{}'.format(int(time.time())) #kita buat nama untuk memudahkan dalam membedakkan dengan model lain karena kita akan buat model lain

tensorboard = TensorBoard(log_dir = 'log/{}'.format(NAME)) #ini adalah call back dari model kita

x = x

y = y

x = x/255.0

model = Sequential()

model.add(Conv2D(64, (3,3), input_shape = x.shape[1:])) #memasukkan layer (konvolusi 2 dimensi(banyak neuron, besar window, besar inputnya))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size = (2,2)))

model.add(Conv2D(64,(3,3)))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size = (2,2))) #mencari nilai terbesar dari windownnya

model.add(Flatten()) #disini kita membuat matriks 3 dimensi menjadi 2 dimensi

model.add(Dense(64))
model.add(Activation('relu')) #perlu diingat kalau habis layer dense perlu activation karena kalau enggak sama aja kaya regression biasa

model.add(Dense(1))
model.add(Activation('sigmoid'))

model.compile(loss = 'binary_crossentropy',
             optimizer = 'adam',
             metrics = ['accuracy'])

model.fit(np.array(x), np.array(y), batch_size = 32, epochs = 10, validation_split = 0.3, callbacks = [tensorboard])



# Menyimpan Model

In [ ]:
model.save('model2_penyakit_cabai.model')

# Melakukan Testing

In [ ]:
import tensorflow as tf
DATADIR_TESTING = "D:\Studi Independetn Orbit\TA\Data Baru\Tes-Cropped"
CATEGORIES_TESTING = ['DaunSehat', 'sakit']
val_data = []
IMG_SIZE = 250

create_training_data(CATEGORIES_TESTING, DATADIR_TESTING, val_data, 250)
model = tf.keras.models.load_model('model2_penyakit_cabai.model')

x_val = [] #training data
y_val = [] #labelnya

for features, label in val_data:
    x_val.append(features)
    y_val.append(label)
    
x_val = np.array(x_val).reshape(-1, IMG_SIZE, IMG_SIZE, 3) 

In [ ]:
y_pred = model.predict(x_val)

In [ ]:
y_pred

In [ ]:
y_val

In [ ]:
def akurasi(y_pred, y_val):
    betul = 0
    salah = 0
    for i in range(len(y_pred)):
        if y_pred[i] == y_val[i]:
            betul+=1
        else:
            salah+=1
    akurasi = betul/len(y_pred)
    return print(akurasi)

In [ ]:
akurasi(y_pred, y_val)

In [ ]:
print(y_pred[0])
plt.imshow(x_val[0])
plt.show()

# Testing

In [ ]:
val_loss, val_acc = model.evaluate(np.array(x_val), np.array(y_val))

# Coba

In [ ]:
import cv2
import tensorflow as tf
import matplotlib.image as mpimg

CATEGORIES = ['DaunSehat', 'sakit']

def change(image):
    IMG_SIZE = 250
    img_array = cv2.imread(image)
    new_array = cv2.resize(img_array,(IMG_SIZE, IMG_SIZE))
    return new_array.reshape(-1, IMG_SIZE, IMG_SIZE, 3)
    
model = tf.keras.models.load_model('model2_penyakit_cabai.model')

def hasil(file):
    gambar = file
    prediction = model.predict([change(gambar)])
    print(prediction)
    img = mpimg.imread(gambar)
    imgplot = plt.imshow(img)
    plt.show();

hasil('DS(94).jpg')

In [ ]:
hasil('KM(99).jpg')

In [ ]:
hasil('VG(76).jpg')

# Mencoba dengan pretrained model VGG16

In [ ]:
from tensorflow.keras.models import Model
import numpy as np
from tensorflow.keras import layers
from tensorflow.keras.applications.vgg16 import VGG16
from tensorflow.keras import layers 
import tensorflow as tf

In [ ]:
training_data_2 = []
IMG_SIZE = 224
create_training_data(CATEGORIES, DATADIR, training_data_2, 224)

base_model = VGG16(input_shape = (224, 224, 3), # Shape of our images
include_top = False, # Leave out the last fully connected layer
weights = 'imagenet')

In [ ]:
import random
random.shuffle(training_data_2)

#disini kita membagi menjadi x dan y nya 
x2 = [] #training data
y2 = [] #labelnya

#memasukkan feature dan label kedalam x dan y
for features, label in training_data_2:
    x2.append(features)
    y2.append(label)

#disini kita perlu mengubah menjadi array karena kalau list enggak bisa masuk ke API Keras
x2 = np.array(x2).reshape(-1, IMG_SIZE, IMG_SIZE, 3) 

In [ ]:
for layer in base_model.layers:
    layer.trainable = False

In [ ]:
# Flatten the output layer to 1 dimension
x = layers.Flatten()(base_model.output)

# Add a fully connected layer with 512 hidden units and ReLU activation
x = layers.Dense(512, activation='relu')(x)

# Add a dropout rate of 0.5
x = layers.Dropout(0.5)(x)

# Add a final sigmoid layer with 1 node for classification output
x = layers.Dense(1, activation='sigmoid')(x)

model2 = tf.keras.models.Model(base_model.input, x)

model2.compile(optimizer = tf.keras.optimizers.RMSprop(lr=0.0001), loss = 'binary_crossentropy',metrics = ['acc'])

In [ ]:
model2.fit(np.array(x2), np.array(y2), epochs=10)

In [ ]:
model.save('VGG16_penyakit_cabai.model')

In [ ]:
import tensorflow as tf
DATADIR_TESTING = "D:\Studi Independetn Orbit\TA\Data Baru\Tes-Cropped"
CATEGORIES_TESTING = ['DaunSehat', 'sakit']
val_data = []
IMG_SIZE = 250

create_training_data(CATEGORIES_TESTING, DATADIR_TESTING, val_data, 250)
model = tf.keras.models.load_model('VGG16_penyakit_cabai.model')

x_val_2 = [] #training data
y_val_2 = [] #labelnya

for features, label in val_data:
    x_val_2.append(features)
    y_val_2.append(label)
    
x_val = np.array(x_val_2).reshape(-1, IMG_SIZE, IMG_SIZE, 3)

In [ ]:
val_loss, val_acc = model2.evaluate(np.array(x_val_2), np.array(y_val_2))

In [ ]:
import cv2
import tensorflow as tf
import matplotlib.image as mpimg

CATEGORIES = ['DaunSehat', 'sakit']

def change(image):
    IMG_SIZE = 250
    img_array = cv2.imread(image)
    new_array = cv2.resize(img_array,(IMG_SIZE, IMG_SIZE))
    return new_array.reshape(-1, IMG_SIZE, IMG_SIZE, 3)
    
model = tf.keras.models.load_model('VGG16_penyakit_cabai.model')

def hasil2(file):
    gambar = file
    prediction = model.predict([change(gambar)])
    print(prediction)
    img = mpimg.imread(gambar)
    imgplot = plt.imshow(img)
    plt.show();

hasil('DS(94).jpg')

In [ ]:
hasil2('KM(99).jpg')

In [ ]:
hasil2('VG(76).jpg')

In [ ]:
hasil2('daun cabe.jpg')